# SEO Audit — PainClinics.com

Analyzes 16 months of Google Search Console data across 6 dimensions:
1. Query Analysis & Clustering
2. Page Performance
3. Indexing & Coverage
4. 404 Recovery
5. "Near Me" & Generic Query Deep Dive
6. Action Items

In [ ]:
import pandas as pd
import numpy as np
import re
import os
from pathlib import Path
from urllib.parse import urlparse

pd.set_option('display.max_columns', 20)
pd.set_option('display.max_colwidth', 80)
pd.set_option('display.width', 200)

AUDIT_DIR = Path('.')
if not (AUDIT_DIR / 'painclinics.com-Performance-on-Search-2026-02-25 - Queries.csv').exists():
    AUDIT_DIR = Path('docs/seo-audit')

print(f'Audit directory: {AUDIT_DIR.resolve()}')
print('Files:', [f.name for f in AUDIT_DIR.iterdir() if f.suffix in ('.csv', '.json')])

In [ ]:
# Load CSVs
queries_file = list(AUDIT_DIR.glob('*Queries*.csv'))[0]
pages_file = list(AUDIT_DIR.glob('*Pages*.csv'))[0]
coverage_file = list(AUDIT_DIR.glob('*Coverage*.csv'))[0]

queries = pd.read_csv(queries_file, dtype_backend='numpy_nullable')
pages = pd.read_csv(pages_file, dtype_backend='numpy_nullable')
coverage = pd.read_csv(coverage_file, dtype_backend='numpy_nullable')

# Normalize CTR from percentage string to float
for df in [queries, pages]:
    df['CTR'] = df['CTR'].astype(str).str.rstrip('%').astype(float)
    df['Clicks'] = pd.to_numeric(df['Clicks'], errors='coerce')
    df['Impressions'] = pd.to_numeric(df['Impressions'], errors='coerce')
    df['Position'] = pd.to_numeric(df['Position'], errors='coerce')

# Rename first column for consistency
queries.rename(columns={queries.columns[0]: 'Query'}, inplace=True)
pages.rename(columns={pages.columns[0]: 'Page'}, inplace=True)

print(f'Queries: {len(queries)} rows')
print(f'Pages: {len(pages)} rows')
print(f'Coverage: {len(coverage)} rows')
print(f'Queries dtypes: {dict(queries.dtypes)}')
print()
queries.head(3)

---
## 1A: Query Analysis & Clustering

In [ ]:
# --- Condition and treatment keyword lists ---
CONDITIONS = [
    'back pain', 'neck pain', 'sciatica', 'neuropathy', 'fibromyalgia',
    'arthritis', 'migraine', 'headache', 'knee pain', 'hip pain',
    'shoulder pain', 'chronic pain', 'joint pain', 'nerve pain',
    'herniated disc', 'spinal stenosis', 'degenerative disc',
    'carpal tunnel', 'plantar fasciitis', 'complex regional',
    'crps', 'radiculopathy', 'myofascial', 'tendonitis',
    'whiplash', 'post surgical pain', 'cancer pain',
    'pelvic pain', 'abdominal pain', 'chest pain',
]

TREATMENTS = [
    'injection', 'nerve block', 'epidural', 'steroid',
    'physical therapy', 'spinal cord stimulator', 'spinal cord stimulation',
    'radiofrequency ablation', 'rfa', 'acupuncture', 'chiropractic',
    'massage therapy', 'regenerative', 'prp', 'platelet rich plasma',
    'stem cell', 'ketamine', 'infusion', 'tens', 'biofeedback',
    'cognitive behavioral', 'medication management', 'opioid',
    'suboxone', 'trigger point', 'facet joint', 'si joint',
    'kyphoplasty', 'vertebroplasty', 'discography', 'intrathecal pump',
    'botox', 'cortisone',
]

# US state names and abbreviations for city-state detection
STATES = [
    'alabama','alaska','arizona','arkansas','california','colorado',
    'connecticut','delaware','florida','georgia','hawaii','idaho',
    'illinois','indiana','iowa','kansas','kentucky','louisiana',
    'maine','maryland','massachusetts','michigan','minnesota',
    'mississippi','missouri','montana','nebraska','nevada',
    'new hampshire','new jersey','new mexico','new york',
    'north carolina','north dakota','ohio','oklahoma','oregon',
    'pennsylvania','rhode island','south carolina','south dakota',
    'tennessee','texas','utah','vermont','virginia','washington',
    'west virginia','wisconsin','wyoming',
    ' al ', ' ak ', ' az ', ' ar ', ' ca ', ' co ', ' ct ', ' de ',
    ' fl ', ' ga ', ' hi ', ' id ', ' il ', ' in ', ' ia ', ' ks ',
    ' ky ', ' la ', ' me ', ' md ', ' ma ', ' mi ', ' mn ', ' ms ',
    ' mo ', ' mt ', ' ne ', ' nv ', ' nh ', ' nj ', ' nm ', ' ny ',
    ' nc ', ' nd ', ' oh ', ' ok ', ' or ', ' pa ', ' ri ', ' sc ',
    ' sd ', ' tn ', ' tx ', ' ut ', ' vt ', ' va ', ' wa ', ' wv ',
    ' wi ', ' wy ',
]

def classify_query(q):
    q_lower = f' {q.lower()} '
    
    if 'painclinics' in q_lower or 'pain clinics directory' in q_lower:
        return 'brand'
    
    if any(kw in q_lower for kw in ['near me', 'near ', 'nearby', 'close to me']):
        return 'near-me'
    
    if any(kw in q_lower for kw in CONDITIONS):
        return 'condition'
    
    if any(kw in q_lower for kw in TREATMENTS):
        return 'treatment'
    
    if any(st in q_lower for st in STATES):
        return 'city-state'
    
    # Check for common clinic-name patterns: "dr ", "clinic", "center", "associates", etc.
    clinic_indicators = ['dr ', 'dr.', 'clinic', 'center', 'associates', 'medical', 'health',
                         'wellness', 'rehab', 'institute', 'group', 'specialists',
                         'physicians', 'care ', 'pllc', 'llc', 'md ']
    if any(kw in q_lower for kw in clinic_indicators):
        return 'clinic-name'
    
    if 'pain management' in q_lower or 'pain doctor' in q_lower or 'pain specialist' in q_lower:
        return 'generic'
    
    return 'generic'

# Ensure numeric types for aggregation (pandas 3.0 uses StringDtype by default)
for col in ['Clicks', 'Impressions', 'Position', 'CTR']:
    queries[col] = pd.to_numeric(queries[col], errors='coerce')

queries['Cluster'] = queries['Query'].apply(classify_query)

print('Query cluster distribution:')
cluster_summary = queries.groupby('Cluster').agg(
    count=('Query', 'size'),
    total_impressions=('Impressions', 'sum'),
    total_clicks=('Clicks', 'sum'),
    avg_position=('Position', 'mean'),
    avg_ctr=('CTR', 'mean'),
).sort_values('total_impressions', ascending=False)
cluster_summary

In [ ]:
# Top queries by impressions (where Google thinks you're relevant)
print('Top 25 queries by impressions:')
queries.nlargest(25, 'Impressions')[['Query', 'Clicks', 'Impressions', 'CTR', 'Position', 'Cluster']]

In [ ]:
# CTR by position bucket
def position_bucket(pos):
    if pos <= 3:
        return '1-3'
    elif pos <= 10:
        return '4-10'
    elif pos <= 20:
        return '11-20'
    else:
        return '21+'

queries['PosBucket'] = queries['Position'].apply(position_bucket)

bucket_stats = queries.groupby('PosBucket').agg(
    queries_count=('Query', 'size'),
    total_impressions=('Impressions', 'sum'),
    total_clicks=('Clicks', 'sum'),
    avg_ctr=('CTR', 'mean'),
).reindex(['1-3', '4-10', '11-20', '21+'])
bucket_stats['weighted_ctr'] = (bucket_stats['total_clicks'] / bucket_stats['total_impressions'] * 100).round(2)

print('CTR by position bucket:')
bucket_stats

In [ ]:
# Quick wins: position 4-15, 50+ impressions — sorted by impressions desc
quick_wins = queries[
    (queries['Position'] >= 4) & 
    (queries['Position'] <= 15) & 
    (queries['Impressions'] >= 50)
].sort_values('Impressions', ascending=False).copy()

print(f'Quick wins: {len(quick_wins)} queries at position 4-15 with 50+ impressions')
quick_wins.head(20)[['Query', 'Clicks', 'Impressions', 'CTR', 'Position', 'Cluster']]

In [ ]:
# Gap analysis: high impressions but position >10 (ranking opportunity)
gaps = queries[
    (queries['Position'] > 10) & 
    (queries['Impressions'] >= 100)
].sort_values('Impressions', ascending=False)

print(f'Ranking opportunities: {len(gaps)} queries with 100+ impressions but position >10')
gaps.head(20)[['Query', 'Clicks', 'Impressions', 'CTR', 'Position', 'Cluster']]

In [ ]:
# CTR anomalies: position <5 but CTR <3% — title/description problems
ctr_anomalies = queries[
    (queries['Position'] < 5) & 
    (queries['CTR'] < 3) &
    (queries['Impressions'] >= 50)
].sort_values('Impressions', ascending=False)

print(f'CTR anomalies: {len(ctr_anomalies)} queries with position <5 but CTR <3%')
ctr_anomalies.head(20)[['Query', 'Clicks', 'Impressions', 'CTR', 'Position', 'Cluster']]

---
## 1B: Page Performance Analysis

In [ ]:
# Classify page types from URL
def classify_page(url):
    path = urlparse(url).path.rstrip('/')
    query = urlparse(url).query
    
    if path == '' or path == '/':
        if 'clinics=' in query:
            return 'clinic-old-url'  # Legacy /?clinics= format
        return 'homepage'
    
    if path.startswith('/pain-management/'):
        parts = path.replace('/pain-management/', '').strip('/').split('/')
        if len(parts) == 1:
            slug = parts[0]
            # State pages are 2-letter abbreviations
            if re.match(r'^[a-z]{2}$', slug):
                return 'state'
            return 'clinic'
        elif len(parts) == 2:
            # state/city
            if re.match(r'^[a-z]{2}$', parts[0]):
                return 'city'
            return 'clinic'
        return 'clinic'
    
    if path.startswith('/blog'):
        return 'blog'
    
    if path in ['/about', '/faq', '/contact', '/privacy', '/terms', '/claim']:
        return 'static'
    
    if path.startswith('/directory') or path.startswith('/search'):
        return 'directory'
    
    return 'other'

# Ensure numeric types for pages too
for col in ['Clicks', 'Impressions', 'Position', 'CTR']:
    pages[col] = pd.to_numeric(pages[col], errors='coerce')

pages['PageType'] = pages['Page'].apply(classify_page)

print('Page type distribution:')
type_summary = pages.groupby('PageType').agg(
    count=('Page', 'size'),
    total_impressions=('Impressions', 'sum'),
    total_clicks=('Clicks', 'sum'),
    avg_position=('Position', 'mean'),
    avg_ctr=('CTR', 'mean'),
).sort_values('total_impressions', ascending=False)
type_summary

In [ ]:
# Top pages by impressions (not clicks) — shows underperforming pages
print('Top 25 pages by impressions:')
pages.nlargest(25, 'Impressions')[['Page', 'Clicks', 'Impressions', 'CTR', 'Position', 'PageType']]

In [ ]:
# Pages with high impressions but low CTR — meta title/description issues
underperforming_pages = pages[
    (pages['Impressions'] >= 500) &
    (pages['CTR'] < 2)
].sort_values('Impressions', ascending=False)

print(f'Underperforming pages: {len(underperforming_pages)} pages with 500+ impressions and CTR <2%')
underperforming_pages.head(15)[['Page', 'Clicks', 'Impressions', 'CTR', 'Position', 'PageType']]

In [ ]:
# Orphaned pages: zero clicks despite some impressions
orphaned = pages[
    (pages['Clicks'] == 0) &
    (pages['Impressions'] >= 10)
].sort_values('Impressions', ascending=False)

print(f'Orphaned pages (0 clicks, 10+ impressions): {len(orphaned)}')
orphaned.head(15)[['Page', 'Impressions', 'Position', 'PageType']]

---
## 1C: Indexing & Coverage Analysis

In [ ]:
# The coverage CSV from GSC has date-level data. Analyze the most recent snapshot.
print('Coverage data columns:', coverage.columns.tolist())
print()

# Parse the date-based coverage data
coverage['Date'] = pd.to_datetime(coverage['Date'])

# Convert numeric columns
for col in ['Indexed', 'Not indexed', 'Impressions']:
    if col in coverage.columns:
        coverage[col] = pd.to_numeric(coverage[col], errors='coerce')

# Get the most recent row with both indexed and not-indexed values
indexed_col = 'Indexed' if 'Indexed' in coverage.columns else None
if indexed_col:
    recent = coverage.dropna(subset=[indexed_col]).tail(5)
    print('Recent coverage snapshots:')
    display(recent)
else:
    print('No "Indexed" column found in coverage data')
    print(coverage.head())

In [ ]:
# Coverage trend
not_indexed_col = 'Not indexed'
indexed_col = 'Indexed'

if indexed_col in coverage.columns and not_indexed_col in coverage.columns:
    coverage_with_data = coverage.dropna(subset=[indexed_col, not_indexed_col])
    if len(coverage_with_data) > 0:
        latest = coverage_with_data.iloc[-1]
        indexed_val = float(latest[indexed_col])
        not_indexed_val = float(latest[not_indexed_col])
        print(f"Latest coverage snapshot ({latest['Date'].strftime('%Y-%m-%d')}):")
        print(f"  Indexed: {indexed_val:.0f}")
        print(f"  Not indexed: {not_indexed_val:.0f}")
        total = indexed_val + not_indexed_val
        if total > 0:
            pct = indexed_val / total * 100
            print(f"  Index rate: {pct:.1f}%")
    else:
        print('No rows with both Indexed and Not indexed values.')
else:
    print('Coverage CSV does not have Indexed/Not indexed breakdown.')
    print('Available columns:', coverage.columns.tolist())
    print()
    print('Raw coverage data sample:')
    print(coverage.head(10))

---
## 1D: 404 Recovery Audit

Cross-reference any known 404 patterns with pages that still receive impressions.

In [ ]:
# Check for legacy /?clinics= URLs still getting impressions
legacy_urls = pages[pages['PageType'] == 'clinic-old-url'].sort_values('Impressions', ascending=False)

print(f'Legacy /?clinics= URLs still getting impressions: {len(legacy_urls)}')
print(f'Total impressions on legacy URLs: {legacy_urls["Impressions"].sum():,}')
print(f'Total clicks on legacy URLs: {legacy_urls["Clicks"].sum():,}')
print()
legacy_urls.head(15)[['Page', 'Clicks', 'Impressions', 'CTR', 'Position']]

In [ ]:
# Pages with www vs non-www — potential duplicate/redirect issues
www_pages = pages[pages['Page'].str.contains('://www.', regex=False)]

print(f'Pages with www prefix: {len(www_pages)}')
if len(www_pages) > 0:
    print(f'Total impressions on www pages: {www_pages["Impressions"].sum():,}')
    www_pages.head(10)[['Page', 'Clicks', 'Impressions', 'CTR', 'Position']]

---
## 1E: "Near Me" & Generic Query Deep Dive

In [ ]:
# All "near me" queries
near_me = queries[queries['Cluster'] == 'near-me'].sort_values('Impressions', ascending=False)

print(f'"Near me" queries: {len(near_me)}')
print(f'Total "near me" impressions: {near_me["Impressions"].sum():,}')
print(f'Total "near me" clicks: {near_me["Clicks"].sum():,}')
print(f'Average position: {near_me["Position"].mean():.1f}')
print()
near_me.head(20)[['Query', 'Clicks', 'Impressions', 'CTR', 'Position']]

In [ ]:
# Condition-specific queries — what conditions drive the most impressions?
condition_queries = queries[queries['Cluster'] == 'condition'].sort_values('Impressions', ascending=False)

print(f'Condition queries: {len(condition_queries)}')
print(f'Total impressions: {condition_queries["Impressions"].sum():,}')
print()
condition_queries.head(20)[['Query', 'Clicks', 'Impressions', 'CTR', 'Position']]

In [ ]:
# Treatment-specific queries
treatment_queries = queries[queries['Cluster'] == 'treatment'].sort_values('Impressions', ascending=False)

print(f'Treatment queries: {len(treatment_queries)}')
print(f'Total impressions: {treatment_queries["Impressions"].sum():,}')
print()
treatment_queries.head(20)[['Query', 'Clicks', 'Impressions', 'CTR', 'Position']]

In [ ]:
# Generic queries (pain management, pain doctor, etc.) — highest-value non-branded
generic_queries = queries[queries['Cluster'] == 'generic'].sort_values('Impressions', ascending=False)

print(f'Generic queries: {len(generic_queries)}')
print(f'Total impressions: {generic_queries["Impressions"].sum():,}')
print()
generic_queries.head(20)[['Query', 'Clicks', 'Impressions', 'CTR', 'Position']]

In [ ]:
# Content gap analysis: treatment/condition queries with no dedicated page
# These queries have high impressions but the site likely ranks with homepage or clinic pages
high_value_nonbrand = queries[
    queries['Cluster'].isin(['near-me', 'condition', 'treatment', 'generic', 'city-state']) &
    (queries['Impressions'] >= 50)
].sort_values('Impressions', ascending=False)

print(f'High-value non-brand queries (50+ impressions): {len(high_value_nonbrand)}')
print(f'Total impressions: {high_value_nonbrand["Impressions"].sum():,}')
print()

# Group by cluster
hv_by_cluster = high_value_nonbrand.groupby('Cluster').agg(
    queries=('Query', 'size'),
    total_impressions=('Impressions', 'sum'),
    avg_position=('Position', 'mean'),
).sort_values('total_impressions', ascending=False)
hv_by_cluster

---
## 1F: Action Items Generator

In [ ]:
action_items = []

# 1. Meta title/description rewrites for CTR anomalies
for _, row in ctr_anomalies.iterrows():
    action_items.append({
        'priority': 'high' if row['Impressions'] >= 200 else 'medium',
        'type': 'meta',
        'query_or_url': row['Query'],
        'action': f'Rewrite meta title/description — position {row["Position"]:.1f} but only {row["CTR"]:.1f}% CTR',
        'impressions': row['Impressions'],
        'expected_impact': 'Improve CTR from current position'
    })

# 2. Quick win content optimization
for _, row in quick_wins.head(30).iterrows():
    if row['Cluster'] in ('near-me', 'condition', 'treatment', 'generic', 'city-state'):
        action_items.append({
            'priority': 'high',
            'type': 'content',
            'query_or_url': row['Query'],
            'action': f'Optimize content for this {row["Cluster"]} query — position {row["Position"]:.1f}, {row["Impressions"]} impressions',
            'impressions': row['Impressions'],
            'expected_impact': 'Move from page 1-2 to top 3'
        })

# 3. Legacy URL redirects
for _, row in legacy_urls.head(20).iterrows():
    slug = urlparse(row['Page']).query.replace('clinics=', '')
    action_items.append({
        'priority': 'high' if row['Impressions'] >= 1000 else 'medium',
        'type': 'redirect',
        'query_or_url': row['Page'],
        'action': f'Redirect legacy /?clinics={slug} to canonical URL — {row["Impressions"]:,} impressions',
        'impressions': row['Impressions'],
        'expected_impact': 'Consolidate ranking signals to canonical URL'
    })

# 4. www redirect consolidation
for _, row in www_pages.head(10).iterrows():
    action_items.append({
        'priority': 'medium',
        'type': 'redirect',
        'query_or_url': row['Page'],
        'action': f'Ensure www → non-www redirect — {row["Impressions"]:,} impressions on www version',
        'impressions': row['Impressions'],
        'expected_impact': 'Consolidate duplicate URLs'
    })

# 5. Treatment content gaps — high impression treatment queries with poor ranking
treatment_gaps = treatment_queries[
    (treatment_queries['Position'] > 15) &
    (treatment_queries['Impressions'] >= 50)
]
for _, row in treatment_gaps.iterrows():
    action_items.append({
        'priority': 'medium',
        'type': 'new-page',
        'query_or_url': row['Query'],
        'action': f'Consider treatment landing page — currently position {row["Position"]:.0f} for {row["Impressions"]} impressions',
        'impressions': row['Impressions'],
        'expected_impact': 'Dedicated content could rank for treatment queries'
    })

# 6. Underperforming pages need meta/content fixes
for _, row in underperforming_pages.head(15).iterrows():
    action_items.append({
        'priority': 'high' if row['Impressions'] >= 2000 else 'medium',
        'type': 'meta',
        'query_or_url': row['Page'],
        'action': f'Improve meta for underperforming page — {row["Impressions"]:,} impressions, {row["CTR"]:.1f}% CTR, position {row["Position"]:.1f}',
        'impressions': row['Impressions'],
        'expected_impact': 'Higher CTR from existing impressions'
    })

actions_df = pd.DataFrame(action_items)
# Sort: high priority first, then by impressions
priority_order = {'high': 0, 'medium': 1, 'low': 2}
actions_df['_priority_sort'] = actions_df['priority'].map(priority_order)
actions_df = actions_df.sort_values(['_priority_sort', 'impressions'], ascending=[True, False]).drop(columns='_priority_sort')

print(f'Total action items: {len(actions_df)}')
print(f'  High priority: {(actions_df["priority"] == "high").sum()}')
print(f'  Medium priority: {(actions_df["priority"] == "medium").sum()}')
print()
actions_df.head(20)

---
## Export Output CSVs

In [ ]:
# Export all output CSVs
out = AUDIT_DIR

# 1. Query clusters
queries[['Query', 'Clicks', 'Impressions', 'CTR', 'Position', 'Cluster', 'PosBucket']].to_csv(
    out / 'query-clusters.csv', index=False
)

# 2. Page performance
pages[['Page', 'Clicks', 'Impressions', 'CTR', 'Position', 'PageType']].to_csv(
    out / 'page-performance.csv', index=False
)

# 3. Quick wins
quick_wins[['Query', 'Clicks', 'Impressions', 'CTR', 'Position', 'Cluster']].to_csv(
    out / 'quick-wins.csv', index=False
)

# 4. Content gaps (high-value non-brand queries)
high_value_nonbrand[['Query', 'Clicks', 'Impressions', 'CTR', 'Position', 'Cluster']].to_csv(
    out / 'content-gaps.csv', index=False
)

# 5. Action items
actions_df.to_csv(out / 'action-items.csv', index=False)

# 6. Index gaps (legacy + www URLs as proxy since full coverage export is date-level only)
index_gap_rows = []
for _, row in legacy_urls.iterrows():
    index_gap_rows.append({'url': row['Page'], 'issue': 'legacy-url-format', 'impressions': row['Impressions']})
for _, row in www_pages.iterrows():
    index_gap_rows.append({'url': row['Page'], 'issue': 'www-duplicate', 'impressions': row['Impressions']})

index_gaps_df = pd.DataFrame(index_gap_rows).sort_values('impressions', ascending=False) if index_gap_rows else pd.DataFrame(columns=['url', 'issue', 'impressions'])
index_gaps_df.to_csv(out / 'index-gaps.csv', index=False)

print('Exported files:')
for f in ['query-clusters.csv', 'page-performance.csv', 'quick-wins.csv', 'content-gaps.csv', 'action-items.csv', 'index-gaps.csv']:
    path = out / f
    if path.exists():
        rows = len(pd.read_csv(path))
        print(f'  {f}: {rows} rows')
    else:
        print(f'  {f}: NOT FOUND')

---
## Summary

Review the exported CSVs and the analysis above. Key things to look for:

1. **Quick wins** — queries at position 4-15 where small content/meta improvements can move you to top 3
2. **CTR anomalies** — good positions but poor click-through rates mean bad titles/descriptions
3. **Near-me gaps** — high-impression generic queries where city/state pages need enrichment
4. **Legacy URL consolidation** — `/?clinics=` URLs still getting impressions need redirects
5. **Treatment content gaps** — treatment queries with no dedicated landing page

Proceed to Step 3 (implementation) based on the action-items.csv priorities.